## Run the following cells for either Experiment Code or Full Run

In [1]:
import numpy as np
import pandas as pd

import unidecode
from datetime import datetime

In [2]:
# configure df options
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 20)

In [3]:
input_path = '../../dataset/bentre'
input_files = {
    'So Lieu Man Ben Tre 2016.xlsx': [2, 5],  # GHoa, MHoa
    'So Lieu Man Ben Tre 2017.xlsx': [4],  # MHoa
    'So Lieu Man Ben Tre 2018.xlsx': [0, 1, 2, 3, 4, 5, 6, 7],  # Vam Giong, Binh Dai, Giao Hoa, An Thuan, Son Doc, My Hoa, Ben Trai, Huong My
}

In [4]:
output_path = '../../dataset/csv/type1'

In [5]:
output_columns = [
    'code',
    'date',
    'min',
    'max',
]

In [6]:
# define method to be applied toward new column 'date'
def calculate_date(row):
    date = None
    try: 
        date = datetime(year=year, month=(row['month'] // 2) + 1, day=row['day'])
    except ValueError:  # handle invalid days of months, such as 02/31
        pass
        
    return date

## Experiment Code

In [7]:
file = list(input_files.keys())[2]

file

'So Lieu Man Ben Tre 2018.xlsx'

In [8]:
sheets = input_files[file]

sheets

[0, 1, 2, 3, 4, 5, 6, 7]

In [10]:
# read excel file
filename = f'{input_path}/{file}'
excel = pd.ExcelFile(filename)

filename

'../../dataset/bentre/So Lieu Man Ben Tre 2018.xlsx'

In [11]:
# read a sheet
df = pd.read_excel(excel, sheet_name=sheets[0])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 11 columns):
ĐÀI KTTV KHU VỰC NAM BỘ    37 non-null object
Unnamed: 1                 34 non-null object
Unnamed: 2                 33 non-null object
BIỂU GHI ĐỘ MẶN (g/l)      32 non-null object
Unnamed: 4                 30 non-null object
Unnamed: 5                 34 non-null object
Unnamed: 6                 34 non-null object
Unnamed: 7                 33 non-null object
Unnamed: 8                 32 non-null object
Unnamed: 9                 35 non-null object
Unnamed: 10                33 non-null object
dtypes: object(11)
memory usage: 3.3+ KB


In [12]:
df.head()

,ĐÀI KTTV KHU VỰC NAM BỘ,Unnamed: 1,Unnamed: 2,BIỂU GHI ĐỘ MẶN (g/l),Unnamed: 4,...,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,ĐÀI KTTV TỈNH TIỀN GIANG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1,Trạm: Vàm Giồng,NaN,NaN,Sông/Kênh: Cửa Tiểu,NaN,...,Tỉnh: Tiền Giang,NaN,NaN,Năm 2018,NaN
2,Tháng,1,NaN,2,NaN,...,NaN,4,NaN,5,NaN
3,Ngày,Max,Min,Max,Min,...,Min,Max,Min,Max,Min
4,1,0,0,2,0,...,0,4.7,0.5,2.4,0.2


In [13]:
# extract station and convert it to station code
station = df.iloc[2][0]
station = station[station.find(':') + 1:].strip()
station = unidecode.unidecode(station).upper().replace(' ', '')

station

'THANG'

In [ ]:
# extract year
year = df.iloc[2][11]
year = int(year[-4:])

year

In [ ]:
# name columns
df.columns = [i for i in range(len(df.columns))]

df.head()

In [ ]:
# drop non-data rows: first 5 ones
df.drop([*range(5)], inplace=True)
    
df.head()

In [ ]:
# merge max + min column pairs into ones for unpivot
for c in range(1, len(df.columns), 2):
    df.iloc[:, c] = df.iloc[:, c + 1].astype('str') + '/' + df.iloc[:, c].astype('str')
    
df.head()

In [ ]:
# drop even columns (min ones)
df.drop([*range(2, len(df.columns), 2)], axis=1, inplace=True)

df.head()

In [ ]:
# unpivot the dataframe from wide (column based) format to long (row based) one
df = df.melt(id_vars=[0], var_name='month', value_name='min/max')

df.head()

In [ ]:
# rename first column
df.rename(columns={ 0: 'day'}, inplace=True)

df.head()

In [ ]:
# remove summary row if any
df.drop(df[df['day'] == 'Max/min'].index, errors='ignore', inplace=True)

df.tail()

In [ ]:
# add station code column
df.insert(0, output_columns[0], station)
    
df.head()

In [ ]:
# calculate 'date' column
df[output_columns[1]] = df.apply(calculate_date, axis=1)

df.head()

In [ ]:
# reconstruct min and max columns
df[[output_columns[2], output_columns[3]]] = df['min/max'].str.split('/', expand=True)

df.head()

In [ ]:
# drop unnecessary columns
df.drop(['day', 'month', 'min/max'], axis=1, inplace=True)

df.head()

In [ ]:
# review invalid date rows
df[df['date'].isnull()]

In [ ]:
# check row counts before dropping
df.info()

In [ ]:
# drop all rows that invalid dates
df.drop(df[df['date'].isnull()].index, inplace=True)

df.info()

In [ ]:
# review head rows
df.head()

In [ ]:
# review tail rows
df.tail()

In [ ]:
# convert min and max columns to float dtype
df['min'] = pd.to_numeric(df['min'], downcast='float', errors='coerce')
df['max'] = pd.to_numeric(df['max'], downcast='float', errors='coerce')

df.info()

In [ ]:
# drop all rows that have less than 3 columns filled out: both min and max are missing
df.dropna(thresh=3, inplace=True)

df.info()

In [ ]:
df.tail()

In [ ]:
# save to csv file
df.to_csv(f'{output_path}/{year}-{station}.csv', index = False)

## Automation Code

In [ ]:
for file in input_files.keys():
    # read excel file
    filename = f'{input_path}/{file}'
    print(f'Processing file: {filename}')
 
    for sheet in input_files[file]:
        print(f'Processing sheet: {sheet}')